In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
TEMPERATURE = 0.7

In [ ]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4', temperature=TEMPERATURE)
llm4_32k = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

In [ ]:
llm3_5.model_name, llm4.model_name, llm4_32k.model_name

In [ ]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=3,
    max_search_expansion=2,
    first_try_temperature=0.,
    max_tokens=10000,
    check_final=True
)

primary_model = 4
sage_model = None

if primary_model == 4:
    primary_model = LangChainLLM(llm4, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000)
    _ = sage_model
    sage_model = primary_model
else:
    primary_model = LangChainLLM(llm3_5, logger=logger)
    sage_model = LangChainLLM(llm4, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000) \
        if sage_model is None or sage_model == 4 else primary_model

executor = Orchestrator(
    config=config,
    llm=primary_model,
    prompts=prompts,
    markdown_logger=logger,
    sage_llm=sage_model,
)

experiment_name = 'example'
# executor.start("""What's x mod (y - z) where x = 10, y = 7, z = 4.""")
executor.start("""
Tao, find a quote from Tao Te Ching which best describe your problem solving philosophy, especially the recursively
generating tree-structure for any problems given to you, starting with a single set of principles and rules.
""")

In [ ]:
backup_chain = executor.chain.copy()

In [ ]:
executor._prompts = PromptDb.load_defaults()

In [ ]:
# executor._chain = executor._chain[:-1]
executor._chain[-1].description

In [ ]:
executor.resume(10000, additional_tokens_for_smarter_llm=500, unblock_initial_expansion=True)

In [ ]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
executor.log_configs(logger)
logger.log_conversation(executor.show_conversation_thread(with_header=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")
